#**PROYECTO FAKE_NEWS**                    
---

###  Carga o Adquisición de Textos
---



In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import spacy
from unidecode import unidecode
import re
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
fake = ('https://drive.google.com/uc?id=111J3U3feYR0ri6_DlP79CGRL-WsmLiF3')
true = ('https://drive.google.com/uc?id=18-_Mg6osJGrLmW3baJTgW_-LWcPCopCE')
df_fake = pd.read_csv(fake, sep=None, engine='python')
df_true = pd.read_csv(true, sep=None, engine='python')

Mounted at /content/drive


In [ ]:
df_fake['type'] = 0

In [ ]:
df_true['type'] = 1

In [ ]:
df = pd.concat([df_fake, df_true]).reset_index(drop=True)

### **Limpieza de los Textos**


In [ ]:
pat = re.compile(r"[^a-z ]")
spaces = re.compile(r"\s{2,}")
def preprocess(text, min_len=1, max_len=23):
    # Normalizamos el texto
    norm_text = unidecode(text).lower()

    # Extraemos tokens
    tokens = norm_text.split()

    # Filtramos palabras por longitud
    filtered_tokens = filter(
            lambda token: (
                len(token) >= min_len and
                len(token) <= max_len
                ),
            tokens
        )
    filtered_text = " ".join(filtered_tokens)
    # Eliminamos caracteres especiales
    clean_text = re.sub(pat, "", filtered_text)
    # Eliminamos espacios duplicados
    spaces_text = re.sub(spaces, " ", clean_text)
    return spaces_text.strip()

In [ ]:
text = df['text'].astype(str)

In [ ]:
text = list(text)

In [ ]:
corpus = list (map(preprocess, text))
#corpus = list(filter(lambda doc: len(doc), corpus))
display(corpus[:10])

['donald trump just couldn t wish all americans a happy new year and leave it at that instead he had to give a shout out to his enemies haters and the very dishonest fake news media the former reality show star had just one job to do and he couldn t do it as our country rapidly grows stronger and smarter i want to wish all of my friends supporters enemies haters and even the very dishonest fake news media a happy and healthy new year president angry pants tweeted will be a great year for america as our country rapidly grows stronger and smarter i want to wish all of my friends supporters enemies haters and even the very dishonest fake news media a happy and healthy new year will be a great year for america donald j trump realdonaldtrump december trump s tweet went down about as welll as you d expectwhat kind of president sends a new year s greeting like this despicable petty infantile gibberish only trump his lack of decency won t even allow him to rise above the gutter long enough to 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['type'], test_size=0.15, random_state=42)
X_train, X_tune, y_train, y_tune = train_test_split(X_train, y_train, test_size=0.1765, random_state=42)

print(f'Numero de registros en train: {len(X_train)}')
print(f'Numero de registros en tune: {len(X_tune)}')
print(f'Numero de registros en test: {len(X_test)}')

Numero de registros en train: 31427
Numero de registros en tune: 6736
Numero de registros en test: 6735


## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
TFIDF_vectorizer = TfidfVectorizer(max_features=512)
TFIDF_vectorizer.fit(X_train)
TFIDFembedding = TFIDF_vectorizer.transform(X_tune)
TFIDFembedding = pd.DataFrame(TFIDFembedding.toarray())
TFIDFembedding.columns = TFIDF_vectorizer.get_feature_names_out()


### **Importar Modelo**
---

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV  #Se importa para realizar partición de conjuntos de datos para entrenamiento, validación y prueba
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.svm import SVC

In [ ]:
# Librerías de apoyo para la visualización de resultados
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

### **Entrenamiento del Modelo y Selección de Hiperparámetros**
---

In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Dividir datos
X = df['text']
y = df['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear pipeline
model = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_df=0.7)),
    ('clf', LogisticRegression(max_iter=1000))
])

# Entrenar modelo
model.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.7, stop_words='english')),
                ('clf', LogisticRegression(max_iter=1000))])

In [ ]:
#Pipeline regresión logistica - TF-IDF
pipeline_lr_TFIDF = Pipeline([('vectorizer', TfidfVectorizer()),
                              ('classifier', LogisticRegression(class_weight="balanced"))])


#Pipeline random forest  - TF-IDF
pipeline_rf_TFIDF = Pipeline([('vectorizer', TfidfVectorizer()),
                        ('classifier', RandomForestClassifier(class_weight="balanced", max_samples= 100))])


In [ ]:
lr_param_grid = {'vectorizer__max_features': [128,256,512],
                 'classifier__C': [0.01, 0.1, 1, 10]}

rf_param_grid = {'vectorizer__max_features': [128,256,512],
                 'classifier__criterion': ['gini', 'entropy'],
                 'classifier__n_estimators': [10,50,100]}

In [ ]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.2/676.2 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 19.3 MB/s eta 0:00:00


In [ ]:
import mlflow
import os
import pandas as pd
from IPython.display import display

In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

In [ ]:
!pip install pyngrok

In [ ]:
token = "2zcN7pTW13NM4QkYXT7ub1oi6i6_39ZNFnm5NsrabQHWoVj3r" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

<NgrokTunnel: "https://7a176d34a081.ngrok-free.app" -> "http://localhost:5000">

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
experiment = mlflow.get_experiment_by_name("Fake_News")
exp_id = experiment.experiment_id if experiment else None

if exp_id is None:
  # Handle the case where the experiment doesn't exist, e.g., create it
  exp_id = mlflow.create_experiment(name="Fake_News", artifact_location="mlruns/")

In [ ]:
#prueba 1

from sklearn.preprocessing import FunctionTransformer

# Wrapper para que sklearn acepte la función
preprocessor = FunctionTransformer(lambda x: list(map(preprocess, x)))

model = Pipeline([
    ('preprocess', preprocessor),
    ('tfidf', TfidfVectorizer(stop_words='english', max_df=0.7)),
    ('clf', LogisticRegression(max_iter=1000))
])

# Entrenar
model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 FunctionTransformer(func=<function <lambda> at 0x79b17aa66660>)),
                ('tfidf', TfidfVectorizer(max_df=0.7, stop_words='english')),
                ('clf', LogisticRegression(max_iter=1000))])

In [ ]:
with mlflow.start_run(
        run_name="LogisticRegression", experiment_id=exp_id
        ):
    model = Pipeline([
    ('preprocess', preprocessor),
    ('tfidf', TfidfVectorizer(stop_words='english', max_df=0.7)),
    ('clf', LogisticRegression(max_iter=1000))
])
    model.fit(X_train, y_train)
    mlflow.sklearn.log_model(model, "model")
    mlflow.log_metric("accuracy", model.score(X_test, y_test))

2025/07/24 01:56:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/24 01:56:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression at: http://localhost:5000/#/experiments/1/runs/d556afe32e8944b6989c9406f8f0bd1a
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [ ]:
model_name = 'FakeN'
model_version = 1
model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")
display(model)

mlflow.pyfunc.loaded_model:
  artifact_path: /content/mlruns/models/m-497bd2c2015f4601a1235043d027ec62/artifacts
  flavor: mlflow.sklearn
  run_id: d556afe32e8944b6989c9406f8f0bd1a

In [ ]:
#Noticia Falsa
text1 = "A group of rogue biotechnologists from a private lab in Switzerland claim to have isolated a gene from the Turritopsis dohrnii jellyfish that can reverse aging in humans. The team allegedly began clandestine human trials in May 2025, with participants reporting “miraculous rejuvenation.” Critics warn that the research is being suppressed by major pharmaceutical companies that profit from aging-related treatments."

In [ ]:
prediction = model.predict([text3])
print(prediction)

[0]


In [ ]:
#Noticia Verdadera
textT1="China is taking a passive yet opportunistic stance toward the U.S. during Trump's second term, deeming his policies self-sabotaging and allowing China to expand its global influence. Analysts liken Trump to Gorbachev, suggesting U.S. decline"

In [ ]:
prediction = model.predict([textT1])
print(prediction)

[1]


In [ ]:
!mkdir src

In [ ]:
%%writefile ./src/mlcli.py
from argparse import ArgumentParser
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")

def main():
    parser = ArgumentParser(
            description="CLI para modelo de detección de Noticias Falsas"
            )
    parser.add_argument("--text", type=str, required=True, help="Texto de la Noticia")
    args = parser.parse_args()
    model = mlflow.pyfunc.load_model("models:/FakeN/1")
    prediction = model.predict([args.text])[0]
    prediction = "Falsa" if prediction else "Verdadera"
    print(f"La Noticia Es : {prediction}")

if __name__ == "__main__":
    main()

Writing ./src/mlcli.py


In [ ]:
!python ./src/mlcli.py --help

usage: mlcli.py [-h] --text TEXT

CLI para modelo de detección de Noticias Falsas

options:
  -h, --help   show this help message and exit
  --text TEXT  Texto de la Noticia


In [ ]:
!python ./src/mlcli.py --text "FBI Russia probe helped by Australian diplomat tip-off: NYT"

La Noticia Es : Verdadera


In [ ]:
!python ./src/mlcli.py --text " Heiress To Disney Empire Knows GOP Scammed Us â€“ SHREDS Them For Tax Bill"

La Noticia Es : Verdadera


In [ ]:
%%writefile ./src/pyproject.toml
[project]
name = "mlcli" # Proyecto Fake News
version = "0.1.0" # versión del proyecto
description = "CLI para modelo de detección de Noticias Falsas" # descripción
authors = [
    {name="John Alexander Mendoza", email="jhondine182@gmail.com"},
    {name="Luis Eduardo Cardona Pazmiño", email="luise90@gmail.com"},
    {name="Edilson Albeiro Molano Bohorquez", email="edilson.molano@gmail.com"},
    ] # autores con correo.

dependencies = [
    "scikit-learn >= 1.6.1",
    "mlflow >= 3.1.4"
    ]

[build-system]
requires = [
        "setuptools", "wheel"
        ]

[project.scripts]
mlcli = "mlcli:main" # cli

Writing ./src/pyproject.toml


In [ ]:
!pip install ./src/

Processing ./src
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mlcli: filename=mlcli-0.1.0-py3-none-any.whl size=1828 sha256=b7d3bebe394509750479304864e8be15b6be9565173e6e978cf4ad8d75ffd6c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-im3076gt/wheels/13/d6/4c/4a5532438203124a9aa7f2b5aecca42bc2dff1e6ba24bd2ef9
Successfully built mlcli


In [ ]:
!mlcli --text "FBI Russia probe helped by Australian diplomat tip-off: NYT"

La Noticia Es : Verdadera


In [ ]:
!mlcli --text "WASHINGTON (Reuters) – On Wednesday, a Democratic spokesperson falsely claimed that aliens had landed near the White House, causing panic across social media platforms.The comment, made during a hastily arranged press conference, followed an unverified video that surfaced late Tuesday showing what appeared to be strange lights over the National Mall."

La Noticia Es : Falsa
